# 一个差分方法的例子（块状中心差分）darcy equation

稀疏矩阵常用函数

|函数|解释|
|:----|:----|
|tocsc()|Return a cope of this matrix in Compressed Sparse Column format|
|tocsr()|Return a cope of this matrix in Compresseed Sparse Row format|
|todense([order,out])|Return a dense matrix representation of this matrix|

In [2]:
import numpy as np
from scipy.sparse import csr_matrix,hstack,vstack

In [3]:
NC = 16
NE = 40
nx = 4
ny = 4

In [5]:
row1 = [val for val in np.arange(NC) for i in range(2)] #array中的元素重复两遍
row1 

[0,
 0,
 1,
 1,
 2,
 2,
 3,
 3,
 4,
 4,
 5,
 5,
 6,
 6,
 7,
 7,
 8,
 8,
 9,
 9,
 10,
 10,
 11,
 11,
 12,
 12,
 13,
 13,
 14,
 14,
 15,
 15]

In [8]:
N1 = np.arange(NE/2-ny)
N2 = np.arange(ny,NE/2)
col1 = [rv for r in zip(N1,N2) for rv in r] #两个array交叉排列.如果列表很大，可以把zip换成itertools.izip
col1

[0.0,
 4.0,
 1.0,
 5.0,
 2.0,
 6.0,
 3.0,
 7.0,
 4.0,
 8.0,
 5.0,
 9.0,
 6.0,
 10.0,
 7.0,
 11.0,
 8.0,
 12.0,
 9.0,
 13.0,
 10.0,
 14.0,
 11.0,
 15.0,
 12.0,
 16.0,
 13.0,
 17.0,
 14.0,
 18.0,
 15.0,
 19.0]

## 附：两个列表的相互交错

In [21]:
from itertools import chain
a = [1, 2, 3]
b = [4, 5, 6]

In [22]:
list(chain.from_iterable(zip(a, b)))# 方法1

[1, 4, 2, 5, 3, 6]

In [23]:
list(chain(*zip(a, b))) #方法二

[1, 4, 2, 5, 3, 6]

In [29]:
def xmerge(a, c):
    alen, clen = len(a), len(c)
    mlen = min(alen, clen)
    for i in range(mlen):
        yield a[i]
        yield c[i]

    if alen > clen:
        for i in range(mlen, alen):
            yield a[i]
    else:
        for i in range(mlen, clen):
            yield c[i]
c = [4, 5, 6, 7, 8, 9]
d = [i for i in xmerge(a, c)]
e = [i for i in xmerge(c, a)]
d

[1, 4, 2, 5, 3, 6, 7, 8, 9]

In [30]:
e

[4, 1, 5, 2, 6, 3, 7, 8, 9]

In [32]:
from matplotlib.cbook import flatten
a = [1, 2, 3]
b = [4, 5, 6]
list(flatten(zip(a,b)))

[1, 4, 2, 5, 3, 6]

**end**

In [9]:
data1 = np.array([-1,1 ]*NC) #[-1, 1] 重复NC遍

In [34]:
A11 = csr_matrix((data1, (row1, col1)),shape = (NC, NE/2)) #生成稀疏矩阵

data1对应矩阵中的非零元，row1代表的是对应的第几行，例如row1 = [0,0,1,2]代表第一行有两个非零元，第一行1个，第二行1个。col1代表与行相对应的列。

In [11]:
N3 = np.arange((NE/2),dtype = np.int).reshape(4,5)[:,:-1].flatten()# 生成有规律断层的数组
N3

array([ 0,  1,  2,  3,  5,  6,  7,  8, 10, 11, 12, 13, 15, 16, 17, 18])

In [12]:
N4 = np.arange((NE/2+1),dtype = np.int)[1:].reshape(4,5)[:,:-1].flatten()
N4

array([ 1,  2,  3,  4,  6,  7,  8,  9, 11, 12, 13, 14, 16, 17, 18, 19])

In [13]:
col2 = [rv for r in zip(N3,N4) for rv in r]
col2

[0,
 1,
 1,
 2,
 2,
 3,
 3,
 4,
 5,
 6,
 6,
 7,
 7,
 8,
 8,
 9,
 10,
 11,
 11,
 12,
 12,
 13,
 13,
 14,
 15,
 16,
 16,
 17,
 17,
 18,
 18,
 19]

In [35]:
A12 = csr_matrix((data1, (row1, col2)),shape = (NC, NE/2))
A13 = csr_matrix((NC, NC),dtype = np.int)

In [37]:
A1 = hstack((A11, A12, A13)) #横向合并

In [42]:
mu = 1
k = 1
rho = 1


In [5]:
#def get_left_matrix(self,nx,ny,NC,NE):
mu =1;
k = 1;
row1 = [val for val in np.arange(NC) for i in range(2)]
N1 = np.arange(NE/2-ny)
N2 = np.arange(ny,NE/2)
col1 = [rv for r in zip(N1,N2) for rv in r]
data1 = np.array([-1,1 ]*NC)
A11 = csr_matrix((data1, (row1, col1)),shape = (NC, NE/2))
N3 = np.arange((NE/2),dtype = np.int).reshape(4,5)[:,:-1].flatten()
N4 = np.arange((NE/2+1),dtype = np.int)[1:].reshape(4,5)[:,:-1].flatten()
col2 = [rv for r in zip(N3,N4) for rv in r]
A12 = csr_matrix((data1, (row1, col2)),shape = (NC, NE/2))
A13 = csr_matrix((NC, NC),dtype = np.int)
A1 = hstack((A11, A12, A13))

row2 = np.arange(4,NE/2)
data2 = mu*nx/k*np.array([1]*(int(NE/2-ny)))
A21 = csr_matrix((data2,(row2,row2)),shape = (NE/2, NE/2))
A22 = csr_matrix((NE/2, NE/2),dtype = np.int)
row3 = [val for val in np.arange(ny,NE/2-ny) for i in range(2)]
N5 = np.arange(NE/2-2*ny)
N6 = np.arange(ny,NE/2-ny)
col3 = [rv for r in zip(N5,N6) for rv in r]
data3 = np.array([-1,1 ]*(int(NE/2-2*ny)))
A23 = csr_matrix((data3,(row3,col3)),shape = (NE/2, NC))
A2 = hstack((A21, A22, A23))

A31 = csr_matrix((NE/2, NE/2),dtype = np.int)
row4 = np.arange((NE/2),dtype = np.int).reshape(4,5)[:,1:].flatten()
A32 = csr_matrix((data2,(row4,row4)),shape = (NE/2, NE/2))
N7 = np.arange((NE/2),dtype = np.int).reshape(4,5)[:,1:-1].flatten()
row5 = [val for val in N7 for i in range(2)]
N8 = np.arange((NC),dtype = np.int).reshape(4,4)[:,:-1].flatten()
N9 = np.arange((NC+1),dtype = np.int)[1:].reshape(4,4)[:,:-1].flatten()
col4 = [rv for r in zip(N8,N9) for rv in r]
data4 = np.array([-1,1 ]*int(NC-nx))
A33 = csr_matrix((data4,(row5,col4)),shape = (NE/2, NC))
A3 = hstack((A31, A32, A33))
A = vstack((A1,A2,A3)).toarray()

   # return A

In [19]:
A33.toarray()


array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, -1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, -1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,

In [20]:
A

array([[-1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ..., -1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [16]:
B = np.matrix(A)

In [17]:
B.I

LinAlgError: Singular matrix